# Product Catalog Agent Ecosystem

## Introduction

# Product Catalog Agent Ecosystem

This project builds a multi-agent AI system using Google's Agent Development Kit (ADK) with Gemini Large Language Model integration. It implements a vendor product catalog agent capable of providing detailed pricing, availability, and specs for various products. The system includes custom function-wrapped tools for querying product information and managing session state. Agents interact seamlessly to deliver accurate and context-aware customer support through conversational interfaces.

The project demonstrates:
- Building and configuring LLM agents with Google ADK.
- Wrapping Python functions as callable tools for agent workflows.
- Managing session state and ephemeral memory for personalized multi-turn dialogues.
- Evaluating agent responses to ensure correctness and improve quality.
- Providing an interactive notebook UI using ipywidgets for real-time user queries.

This framework is ideal for prototyping AI-driven product support systems and multi-agent orchestration in cloud or notebook environments.

This command installs the Google Agent Development Kit (ADK) Python package, **including support for Agent-to-Agent (A2A) protocol features**.  
- `google-adk` is a framework for developing, orchestrating, and deploying AI agents.
- The `[a2a]` part ensures optional A2A modules are included, allowing your agents to communicate and collaborate with other agents using standardized network APIs.

Use this command in Jupyter or Kaggle notebooks to set up your agent projects with remote protocol support.

In [1]:
%pip install -q google-adk[a2a]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.10.0 wh


**Libraries**

- Imports standard Python libraries for system access, JSON handling, timing, logging, asynchronous execution, HTTP requests, subprocess handling, and type hints.
- Imports core modules from the Google Agent Development Kit (ADK), including:
  - `FunctionTool`: for wrapping Python functions as agent tools.
  - `LlmAgent`, `Gemini`: for building language model agents using Gemini.
  - `InMemorySessionService`: for simple session state management during conversation.
  - `Runner`: for orchestrating agent execution with tools and sessions.
  - `types`: for managing structured agent input/output.
- Imports Kaggle's secrets management utility to handle sensitive credentials in a notebook environment.
- The print statement confirms all necessary libraries have been imported, laying the foundation for building and running agent projects within your notebook.

In [2]:
import os
import json
import time
import logging
import asyncio
import requests
import subprocess
from typing import Dict, Any

from google.adk.tools import FunctionTool
from google.adk.agents import LlmAgent
from google.adk.models import Gemini
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types
from kaggle_secrets import UserSecretsClient

print("✅Installed Necessary libraries")

✅Installed Necessary libraries



**Configure logging for observability**

- Sets up Python’s logging system so that all log messages will include a timestamp (`asctime`), the severity level (`levelname`), and the message itself.
- The log level is set to `INFO`, so all messages at INFO level and above will be shown.
- Retrieves a named logger called `"EnterpriseAgentEcosystem"` which you can use throughout your project to record system events, errors, and diagnostics in a standardized and timestamped format.
- This logging setup is ideal for monitoring and debugging complex agent workflows or applications.


In [3]:
# Configure logging for observability
logging.basicConfig(
    format='%(asctime)s %(levelname)s %(message)s',
    level=logging.INFO
)
logger = logging.getLogger("EnterpriseAgentEcosystem")


**Configuration**

- Sets up key configuration variables for the agent project.
- Retrieves `GOOGLE_API_KEY` from the operating system's environment variables (using `os.environ.get`). If not set, a default key value is used for demonstration.
- Assigns a default user ID, application name, and port for serving the product catalog.
- By using `os.environ.get`, the code supports flexible and secure configuration, making it easy to override values for different development or deployment environments, and avoids hardcoding sensitive data directly in the codebase.


In [4]:
# --- Configuration ---
GOOGLE_API_KEY = os.environ.get("GOOGLEAPIKEY", "GOOGLEAPIKEY")
USER_ID = "default-user"
APP_NAME = "EnterpriseAgentEcosystem"
PRODUCT_CATALOG_PORT = 8001


**Retry Configuration**

- Sets how your agent or service should automatically retry HTTP requests when certain errors occur.
- Uses `types.HttpRetryOptions` to specify:
  - `attempts=5`: Maximum of 5 retry attempts per request.
  - `exp_base=7`: Sets exponential backoff base to increase delay between each retry.
  - `initial_delay=1`: Waits 1 second before the first retry.
  - `http_status_codes=[429, 500, 503, 504]`: Only retries on these common server or rate-limit error codes.
- This configuration helps your agent handle temporary network or service outages gracefully, improving reliability in production or cloud environments[web:61][web:65].

In [5]:
# --- Retry Configuration ---
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

## Agent Setup


**Product catalog**

- Defines a `product_catalog` dictionary containing product names and detailed descriptions, simulating a simple in-memory catalog for an agent or application.
- Implements the `get_product_info` function, which looks up the requested product, returns its details if available, or provides an error message listing available products if not found.
- Wraps this function as an agent tool using `FunctionTool`, making it callable and usable by agents in the Google ADK framework for structured product information queries, recommendations, or workflow automation.

In [6]:
# --- Product catalog ---
from google.adk.tools import FunctionTool

product_catalog = {
    "iphone 15 pro": "iPhone 15 Pro, $999, Low Stock (8 units), 128GB, Titanium finish",
    "samsung galaxy s24": "Samsung Galaxy S24, $799, In Stock (31 units), 256GB, Phantom Black",
    "dell xps 15": "Dell XPS 15, $1299, In Stock (45 units), 15.6' display, 16GB RAM, 512GB SSD",
    "macbook pro 14": "MacBook Pro 14, $1999, In Stock (22 units), M3 Pro chip, 18GB RAM, 512GB SSD",
    "sony wh-1000xm5": "Sony WH-1000XM5 Headphones, $399, In Stock (67 units), Noise-canceling, 30hr battery",
    "ipad air": "iPad Air, $599, In Stock (28 units), 10.9' display, 64GB",
    "lg ultrawide 34": "LG UltraWide 34 Monitor, $499, Out of Stock, Expected next week",
}

def get_product_info(product_name: str) -> str:
    key = product_name.lower().strip()
    if key in product_catalog:
        return f"Product details: {product_catalog[key]}"
    else:
        available = ", ".join([p.title() for p in product_catalog.keys()])
        return f"Sorry, product '{product_name}' not found. Available products: {available}."

get_product_info_tool = FunctionTool(
    func=get_product_info
)

## Product Catalog Tool


Creates a large language model agent using the Google ADK system and the Gemini language model.  
The agent is named `product_catalog_agent` and is described and instructed to behave as a product catalog specialist for an external vendor.  
It is equipped with the `get_product_info_tool`, giving it access to product details, prices, stock status, and specifications from the in-memory product catalog.  
This agent is now ready to answer catalog queries accurately and serve as a specialized assistant in multi-agent workflows or customer support environments.

In [7]:
product_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite"),
    name="product_catalog_agent",  # underscores only, no dashes
    description="External vendor product catalog agent providing product info and availability.",
    instruction=(
        "You are a product catalog specialist from an external vendor. "
        "Use the get_product_info tool to fetch data from the catalog. "
        "Provide accurate product info including price, availability, and specs."
    ),
    tools=[get_product_info_tool]
)


Imports key components from the Google Agent Development Kit (ADK) to enable building and running AI agents:
- `LlmAgent`: The core class for creating a language model agent.
- `Gemini`: Used to specify and configure the Gemini LLM backend for agent generation.
- `FunctionTool`: Lets you wrap custom Python functions as callable tools for agent workflows.
- Sets up the Python logging system to capture info-level diagnostic events, errors, and agent activity for better observability while running or debugging agent-based applications.

In [8]:
from google.adk.agents import LlmAgent
from google.adk.models import Gemini
from google.adk.tools import FunctionTool
import logging

logging.basicConfig(level=logging.INFO)


Defines a function `get_product_info` that looks up a product name in an in-memory catalog and logs the query.  
If found, it returns detailed product information; if not, it returns an apology and lists available products.  
The function is wrapped as a `FunctionTool` that enables it to be invoked by agents.  
An LLM agent named `product_catalog_agent` is created using Gemini, with instructions to act as a product catalog specialist using the tool.  
This agent can now field queries about products, retrieving accurate pricing and availability information, forming a useful component of a multi-agent AI system.

In [9]:
# Function to query the catalog
def get_product_info(product_name: str) -> str:
    key = product_name.lower().strip()
    logging.info(f"Received product info request: {product_name}")
    if key in product_catalog:
        return f"Product details: {product_catalog[key]}"
    else:
        available = ", ".join([p.title() for p in product_catalog.keys()])
        return f"Sorry, product '{product_name}' not found. Available products: {available}."

# Wrap with FunctionTool
get_product_info_tool = FunctionTool(func=get_product_info)

# Define the agent (no retry_options and valid name)
product_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite"),
    name="product_catalog_agent",
    description="Vendor product catalog agent providing product info and availability.",
    instruction=(
        "You are a product catalog specialist. Use get_product_info tool."
    ),
    tools=[get_product_info_tool]
)


- Calls the underlying Python function wrapped by `FunctionTool` directly with test queries.
- The first call queries information for "iphone 15 pro" and will return the product details from the catalog.
- The second call queries a non-existent product, returning an error message listing available product names.
- This demonstrates how to test the tool function independently from the agent, verifying it works as expected before integrating into an agent workflow.

In [10]:
# Example: Direct call to the tool (for testing)
print(get_product_info_tool.func("iphone 15 pro"))
print(get_product_info_tool.func("nonexistent product"))

Product details: iPhone 15 Pro, $999, Low Stock (8 units), 128GB, Titanium finish
Sorry, product 'nonexistent product' not found. Available products: Iphone 15 Pro, Samsung Galaxy S24, Dell Xps 15, Macbook Pro 14, Sony Wh-1000Xm5, Ipad Air, Lg Ultrawide 34.



This code defines two custom tools for managing user information in the session state during agent interactions:

- `save_user_info` stores the user's name and country into the current session's state, allowing persistence of user-specific data across interactions.
- `retrieve_user_info` fetches the stored user name and country from the session, returning default values if none exist.
- Both functions are wrapped as `FunctionTool`s so they can be invoked by the agent framework.
- The `customer_support_agent` is an LLM agent using the Gemini model, configured to use these tools for saving and retrieving user information, enabling personalized, stateful conversations.

In [11]:
# --- Custom Session State Tools ---
def save_user_info(toolcontext, username: str, country: str):
    toolcontext.state["user_username"] = username
    toolcontext.state["user_country"] = country
    return {"status": "success"}

def retrieve_user_info(toolcontext):
    username = toolcontext.state.get("user_username", "Unknown")
    country = toolcontext.state.get("user_country", "Unknown")
    return {"username": username, "country": country}

save_user_info_tool = FunctionTool(func=save_user_info)
retrieve_user_info_tool = FunctionTool(func=retrieve_user_info)

# --- Customer Support Agent ---
customer_support_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite"),
    name="customer_support_agent",
    description="Customer support agent that retrieves and saves user info.",
    instruction="Use save_user_info to record user name/country, retrieve_user_info to fetch.",
    tools=[save_user_info_tool, retrieve_user_info_tool]
)

## Session Management


Imports and initializes an in-memory memory service from the Google Agent Development Kit (ADK).  
This service stores session information and agent memories directly in the application's RAM, enabling quick, ephemeral storage and retrieval of conversational context or knowledge snippets during an agent's runtime.  
Ideal for prototyping, testing, or scenarios where persistence across application restarts is not required.  
It enables agents to maintain state and recall prior interactions within a session, improving the quality and coherence of multi-turn conversations.

In [12]:
from google.adk.memory import InMemoryMemoryService

memory_service = InMemoryMemoryService()


Initializes services and runner for managing agent conversations and state in-memory during runtime:

- `InMemorySessionService` stores session information such as conversation history and state **temporarily in application memory**, supporting multi-turn dialogue management without persistent storage.
- `InMemoryMemoryService` manages ephemeral agent memory, allowing the agent to recall conversation context during the session for better response continuity.
- `APP_NAME` assigns a logical application identifier used across sessions and logging.
- `Runner` orchestrates the interaction between the configured agent (`customer_support_agent`), session services, and the application, handling input/output and maintaining session flow during user interactions.

This setup is ideal for prototyping or local development where persistence across restarts is unnecessary, enabling responsive, stateful conversational AI experiences within a contained environment.

In [13]:
# --- Session and Runner Setup ---
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()
APP_NAME = "EnterpriseAgentDemo"

runner = Runner(agent=customer_support_agent, app_name=APP_NAME, session_service=session_service)

## Evaluation

**Evaluation Intergration**


Defines configuration and logic for evaluating agent responses:  
- `create_evaluation_config()` writes a JSON config that specifies evaluation criteria including tool usage accuracy (`tool_trajectory_avg_score`) and response quality (`response_match_score`).  
- `evaluate_agent_responses()` is a simple function scoring each response 1.0 if it contains the phrase "Product details", else 0.0, to simulate correctness checking.  
- Logs evaluation progress and returns the average score across all responses, supporting systematic quality measurement for agent testing and improvement.  
- This can be integrated into testing pipelines or manual review workflows to ensure agent reliability and accuracy across scenarios.


In [14]:
# --- Evaluation Integration ---

def create_evaluation_config():
    eval_config = {
        "criteria": {
            "tool_trajectory_avg_score": 1.0,
            "response_match_score": 0.8
        },
        "description": "Evaluation checks tool usage correctness and response quality."
    }
    with open("eval_config.json", "w") as f:
        json.dump(eval_config, f, indent=2)
    logger.info("Evaluation configuration created.")

create_evaluation_config()

def evaluate_agent_responses(responses):
    """
    Dummy evaluation: checks if all responses contain expected keywords.
    """
    logger.info("Starting evaluation of agent responses.")
    scores = []
    for response in responses:
        score = 1.0 if "Product details" in response else 0.0
        scores.append(score)
        logger.info(f"Response: {response[:50]}... Score: {score}")

    avg_score = sum(scores)/len(scores) if scores else 0.0
    logger.info(f"Average evaluation score: {avg_score}")
    return avg_score


This code defines a simple synchronous demonstration function `run_demo` that:
- Sends a list of predefined queries to the product information tool by calling its underlying function directly.
- Prints the agent's textual responses to each query, demonstrating the tool's ability to return product details or an error message if the product is not found.
- Collects all responses and passes them to an evaluation function which scores the responses based on predefined criteria.
- Prints the average evaluation score, providing a simple metric for measuring the correctness and quality of the agent's outputs.
- Runs the demo function immediately, showcasing a complete interaction and evaluation cycle in a straightforward way suitable for debugging or prototyping.

In [15]:
# --- Interaction and observability demonstration ---

# Synchronous demo: directly use tool or agent functions
def run_demo():
    queries = [
        "iphone 15 pro",
        "lg ultrawide 34",
        "nonexistent product"
    ]
    responses = []
    # Here, get_product_info_tool is used directly 
    for query in queries:
        response = get_product_info_tool.func(query)
        print(f"Agent: {response}")
        responses.append(response)

    # Evaluate responses
    avg_score = evaluate_agent_responses(responses)
    print(f"\nAverage evaluation score: {avg_score}")

run_demo()

Agent: Product details: iPhone 15 Pro, $999, Low Stock (8 units), 128GB, Titanium finish
Agent: Product details: LG UltraWide 34 Monitor, $499, Out of Stock, Expected next week
Agent: Sorry, product 'nonexistent product' not found. Available products: Iphone 15 Pro, Samsung Galaxy S24, Dell Xps 15, Macbook Pro 14, Sony Wh-1000Xm5, Ipad Air, Lg Ultrawide 34.

Average evaluation score: 0.6666666666666666


## Interactive UI


This code creates a simple interactive UI in a Jupyter or Kaggle notebook using `ipywidgets`:

- A text input box labeled "Product:" is displayed, initially containing the value `'iphone 15 pro'`.
- When the user types a product name and presses Enter, the `on_submit` callback is triggered.
- The callback clears any previous output and calls the `get_product_info_tool` function with the entered product name.
- The agent's response is then printed below the input box inside the output widget.
- This provides a user-friendly way to query the product catalog tool directly from the notebook’s UI, enabling quick testing and interaction without writing code repeatedly.

In [16]:
import ipywidgets as widgets
from IPython.display import display

product_input = widgets.Text(
    value='iphone 15 pro',
    placeholder='Type product name',
    description='Product:',
)

output = widgets.Output()

def on_submit(change):
    output.clear_output()
    response = get_product_info_tool.func(product_input.value)
    with output:
        print("Agent:", response)

product_input.on_submit(on_submit)

display(product_input, output)

/tmp/ipykernel_13/3750435651.py:18: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  product_input.on_submit(on_submit)


Text(value='iphone 15 pro', description='Product:', placeholder='Type product name')

Output()